In [26]:
import numpy as np
import pandas as pd

from collections import Counter

In [53]:
def build_tree(instances, labels, current_depth, max_depth):
    # Recursive algorithm that builds the DecisionTeee
    
    gain, test = find_gain_and_test_to_split(instances, labels)
    
    # How do we stop?  max_depth exceeded or no gain
    if gain == 0 or max_depth <= current_depth:
        return Leaf(labels)
    
    else: 
        # It is worth to split the set of instances in two
        instances_approved, labels_approved, instances_rejected, labels_rejected = \
        split_instances_and_labels_by_test(test, instances, labels)
        # this two pairs will be the left and right sub tree respectively

        # Build each branch in a recursive way
        left_sub_tree = build_tree(instances_approved, labels_approved, current_depth+1, max_depth)
        right_sub_tree   = build_tree(instances_rejected, labels_rejected, current_depth+1, max_depth)
        
        # returns a DecisionNode with the applied test and its left/right computed branches  
        return DecisionNode(test, left_sub_tree, right_sub_tree)

In [341]:
class Leaf:
    """
    Holds the sum of instances for each class
    #  ie, {'Yes': 2, 'No': 2}
    """
    def __init__(self, labels):
        self.counter = dict(Counter(labels))


class DecisionNode:
    """
    Holds a question which can split instances and the respective yes (left branch) no (right branch) splitted instances. 
    """
    def __init__(self, test, left_sub_tree, right_sub_tree):
        self.test = test
        self.left_sub_tree = left_sub_tree
        self.right_sub_tree = right_sub_tree
        
class Test:
    """
    A yes-no test that to apply 
    """
    def __init__(self, feature, value):
        self.feature = feature
        self.value = value
    
    def apply_on_instance(self, _instance):
        """
        Returns true if instance pass the test.
        """
        #print("type {} ".format(type(_instance)))
        return _instance.__getattribute__(self.feature) == self.value
    
    def __repr__(self):
        return "¿Is value for {} equals to {}?".format(self.feature, self.value)

In [342]:
def gini(labels):
    total = np.sum(list(labels.values()))
    
    gini = 1
    
    for k,v in labels.items():
        gini -= (v/total)**2
        
    return gini

def gini_gain(labels, labels_left_sub_tree, labels_right_sub_tree):
    initial_gain = gini(dict(Counter(labels)))
    
    left_sub_tree_gini = gini(dict(Counter(labels_left_sub_tree))) 
    pondered_left_sub_tree_gini = left_sub_tree_gini * len(labels_left_sub_tree) / len(labels)
    
    right_sub_tree_gini = gini(dict(Counter(labels_right_sub_tree)))
    pondered_right_sub_tree_gini = right_sub_tree_gini * len(labels_right_sub_tree) / len(labels)
    
    return initial_gain - pondered_left_sub_tree_gini  - pondered_right_sub_tree_gini 


def split_instances_and_labels_by_test(test, instances, labels):
    _mask_passed = np.array([test.apply_on_instance(_instance) for _instance in instances.itertuples()])
    
    instances_approved, labels_approved, instances_rejected, labels_rejected = \
    instances[_mask_passed], labels[_mask_passed],instances[~_mask_passed], labels[~_mask_passed]
    
    return instances_approved, labels_approved, instances_rejected, labels_rejected

In [343]:
def find_gain_and_test_to_split(instances, labels):
    """
    Finds the 'best' feature and value to split the set of instances decreasing Gini impurity or maximizing Gini Gain
    """
    max_gain = 0
    best_test = None

    for _column in instances.columns:
        for value in set(instances[_column]):
            # Tries each cut over (feature, value)
            _test = Test(_column, value)
            _, labels_left_sub_tree, _, labels_right_right_tree = \
            split_instances_and_labels_by_test(_test, instances, labels)
   
            _gain = gini_gain(labels, labels_left_sub_tree, labels_right_right_tree)
            
            if _gain > max_gain:
                max_gain = _gain
                best_test = _test
            
    return max_gain, best_test


def plot_tree(tree, spacing=""):
    if isinstance(tree, Leaf):
        print (spacing + "Hoja:", tree.counter)
        return

    print (spacing + str(tree.test))

    print (spacing + '--> True:')
    plot_tree(tree.left_sub_tree, spacing + "  ")

    print (spacing + '--> False:')
    plot_tree(tree.right_sub_tree, spacing + "  ")

In [344]:
X = pd.DataFrame([["Sol","Calor","Alta","Debil"],
                ["Sol","Calor","Alta","Fuerte"],
                ["Nublado","Calor","Alta","Debil"],
                ["Lluvia","Templado","Alta","Debil"],
                ["Lluvia","Frio","Normal","Debil"],
                ["Lluvia","Frio","Normal","Fuerte"],
                ["Nublado","Frio","Normal","Fuerte"],
                ["Sol","Templado","Alta","Debil"],
                ["Sol","Frio","Normal","Debil"],
                ["Lluvia","Templado","Normal","Debil"],
                ["Sol","Templado","Normal","Fuerte"],
                ["Nublado","Templado","Alta","Fuerte"],
                ["Nublado","Calor","Normal","Debil"],
                ["Lluvia","Templado","Alta","Fuerte"]],
                columns = ['Cielo', 'Temperatura', 'Humedad', 'Viento'])

y = np.array(['No', 'No', 'Si', 'Si', 'Si', 'No', 'Si', 'No', 'Si', 'Si', 'Si', 'Si', 'Si', 'No']);

#display(X)
#display(y)

In [353]:
tree = build_tree(X, y, 0, 5)
plot_tree(tree)

¿Is value for Cielo equals to Nublado?
--> True:
  Hoja: {'Si': 4}
--> False:
  ¿Is value for Humedad equals to Alta?
  --> True:
    ¿Is value for Cielo equals to Sol?
    --> True:
      Hoja: {'No': 3}
    --> False:
      ¿Is value for Viento equals to Fuerte?
      --> True:
        Hoja: {'No': 1}
      --> False:
        Hoja: {'Si': 1}
  --> False:
    ¿Is value for Viento equals to Fuerte?
    --> True:
      ¿Is value for Cielo equals to Sol?
      --> True:
        Hoja: {'Si': 1}
      --> False:
        Hoja: {'No': 1}
    --> False:
      Hoja: {'Si': 3}


## Resultado esperado

```
¿Es el valor para Cielo igual a Nublado?
--> True:
  ¿Es el valor para Temperatura igual a Frio?
  --> True:
    Hoja: {'Si': 1}
  --> False:
    ¿Es el valor para Temperatura igual a Templado?
    --> True:
      Hoja: {'Si': 1}
    --> False:
      Hoja: {'Si': 2}
--> False:
  ¿Es el valor para Humedad igual a Normal?
  --> True:
    ¿Es el valor para Viento igual a Fuerte?
    --> True:
      Hoja: {'No': 1, 'Si': 1}
    --> False:
      ¿Es el valor para Cielo igual a Sol?
      --> True:
        Hoja: {'Si': 1}
      --> False:
        Hoja: {'Si': 2}
  --> False:
    ¿Es el valor para Cielo igual a Sol?
    --> True:
      ¿Es el valor para Temperatura igual a Templado?
      --> True:
        Hoja: {'No': 1}
      --> False:
        Hoja: {'No': 2}
    --> False:
      Hoja: {'Si': 1, 'No': 1}
```

## Parte 2 (opcional)
Protocolo sklearn para clasificadores. Completar el protocolo requerido por sklearn. Deben completar la función predict utilizando el árbol para predecir valores de nuevas instancias. 


In [354]:
def predict(arbol, x_t):
    _node = arbol
    while type(_node) is not Leaf:
        _node = _node.left_sub_tree if _node.test.apply_on_instance(x_t) else _node.right_sub_tree
    
    amount_instances_in_node = np.sum(list(_node.counter.values()))
    amount_instances_first_class = list(_node.counter.values())[0]
    classes_names = list(_node.counter.keys())
    
    _random_value = np.random.uniform(0, amount_instances_in_node)
    
    return classes_names[0] if _random_value <= amount_instances_first_class else classes_names[1]
        
class MyDecisionTreeClassifier(): 
    def __init__(self, max_depth=10):
        self.tree = None
        self.features = ['Cielo', 'Temperatura', 'Humedad', 'Viento']
        self.max_depth = max_depth
    
    def fit(self, X_train, y_train):
        self.tree = build_tree(pd.DataFrame(X_train, columns=self.features), y_train, 0, self.max_depth)
        return self
    
    def predict(self, X_test):
        predictions = []
        _df = pd.DataFrame(X_test, columns=self.features)
        for x_t in _df.itertuples():
            prediction = predict(self.tree, x_t) 
            print(x_t, "prediction ->", prediction)
            predictions.append(prediction)
        return predictions
    
    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        
        accuracy = sum(y_i == y_j for (y_i, y_j) in zip(y_pred, y_test)) / len(y_test)
        return accuracy
        

# Ejemplo de uso
clf = MyDecisionTreeClassifier()

# Tomar en cuenta que sklearn espera numpy arrays:
clf.fit(np.array(X), y)
clf.score(np.array(X), y);

Pandas(Index=0, Cielo='Sol', Temperatura='Calor', Humedad='Alta', Viento='Debil') prediction -> No
Pandas(Index=1, Cielo='Sol', Temperatura='Calor', Humedad='Alta', Viento='Fuerte') prediction -> No
Pandas(Index=2, Cielo='Nublado', Temperatura='Calor', Humedad='Alta', Viento='Debil') prediction -> Si
Pandas(Index=3, Cielo='Lluvia', Temperatura='Templado', Humedad='Alta', Viento='Debil') prediction -> Si
Pandas(Index=4, Cielo='Lluvia', Temperatura='Frio', Humedad='Normal', Viento='Debil') prediction -> Si
Pandas(Index=5, Cielo='Lluvia', Temperatura='Frio', Humedad='Normal', Viento='Fuerte') prediction -> No
Pandas(Index=6, Cielo='Nublado', Temperatura='Frio', Humedad='Normal', Viento='Fuerte') prediction -> Si
Pandas(Index=7, Cielo='Sol', Temperatura='Templado', Humedad='Alta', Viento='Debil') prediction -> No
Pandas(Index=8, Cielo='Sol', Temperatura='Frio', Humedad='Normal', Viento='Debil') prediction -> Si
Pandas(Index=9, Cielo='Lluvia', Temperatura='Templado', Humedad='Normal', Vient